In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
organism_group_labels = ["Bacteria", "Fungi", "Root-feeding Nematodes", "Bacterivorous Nematodes", "Fungivorous Nematodes", "Omnivorous Nematodes", "Fungivorous Mites", "Omnivorous Mites", "Collembolans"]
filenames = ['random_1', 'random_2', 'random_3', 'random_4', 'random_5', 'random_6', 'clustered_1', 'clustered_2', 'clustered_3', 'clustered_4', 'clustered_5', 'clustered_6']

In [ ]:
def compute_d_index_global(data, grid_dimensions, range_by_type):
    X, Y = grid_dimensions
    num_types = 9
    type_count = np.zeros((X, Y, num_types))  # Counts for each type at each spatial unit
    D_matrices = [np.zeros((X, Y)) for _ in range(num_types)]  # 9 matrices for each type

    # Populate the 2D grid with agent counts
    for _, agent in data.iterrows():
        x, y, t = agent['x'], agent['y'], agent['type']
        type_count[x][y][t] += 1

    # Calculate the total number of agents for each type in the entire grid
    total_count_by_type = np.sum(type_count, axis=(0, 1))

    def compute_neighborhood_counts(x, y, r):
        """Compute counts of each type in the Von Neumann neighborhood of (x, y) with range r."""
        neighborhood_count = np.zeros(num_types)
        for dx in range(-r, r + 1):
            for dy in range(-r + abs(dx), r - abs(dx) + 1):
                    nx = (x + dx) % X
                    ny = (y + dy) % Y
                    neighborhood_count += type_count[nx][ny]
        return neighborhood_count

    # Calculate the Dissimilarity Index for each spatial unit
    for x in range(X):
        for y in range(Y):
            for t in range(num_types):
                r = range_by_type[t]
                neighborhood_count = compute_neighborhood_counts(x, y, r)
                
                for t_prime in range(num_types):
                    if neighborhood_count[t] > 0 and total_count_by_type[t] > 0 and total_count_by_type[t_prime] > 0:
                        prop_t = neighborhood_count[t] / total_count_by_type[t]
                        prop_t_prime = neighborhood_count[t_prime] / total_count_by_type[t_prime]
                        
                        # Calculate the absolute difference
                        D = abs(prop_t - prop_t_prime)
                        
                        # Accumulate the difference in the respective matrix
                        D_matrices[t][x, y] += D

    og_matrix = D_matrices
    # Normalize the D-index matrices
    for t in range(num_types):
        D_max = np.max(D_matrices[t])
        if D_max > 0:
            D_matrices[t] /= D_max  # Normalize to [0, 1]
        else:
            D_matrices[t] = np.zeros_like(D_matrices[t])  # Handle cases with no data

    return D_matrices, og_matrix

In [ ]:
indices = pd.DataFrame(columns=['filename', 'init_locs', 'seed', 'type_id', '0', '1', '2', '3', '4', '5', '6', '7', '8'])
d_indices = []
for idx, filename in enumerate(filenames):  
    df = pd.read_csv(path + filename + ".csv")
    data = df[df["tick"] == sample_times[idx]]
    data = data.reset_index(drop=True)
    d_index, og_ds = compute_d_index_global(data, (400, 400), [1,1,3,3,3,4,5,6,6])
    for type_id, row in enumerate(d_index):
        indices = pd.concat([indices, pd.DataFrame({'filename': [filename], 'init_locs': [init_locs[idx]], 'seed': [init_seed[idx]], 'type_id': [type_id], '0': [row[0]], '1': [row[1]], '2': [row[2]], '3': [row[3]], '4': [row[4]], '5': [row[5]], '6': [row[6]], '7': [row[7]], '8': [row[8]]})])
    d_indices.append(d_index)

indices.to_csv('known_d_indices_global_r_org.csv', index=False)
arr = np.asarray(d_indices)
np.save('d_indices_global_r_org.npy', arr)

In [ ]:
indices = pd.DataFrame(columns=['filename', 'init_locs', 'seed', 'type_id', '0', '1', '2', '3', '4', '5', '6', '7', '8'])
d_indices = []
for idx, filename in enumerate(filenames):  
    df = pd.read_csv(path + filename + ".csv")
    data = df[df["tick"] == sample_times[idx]]
    data = data.reset_index(drop=True)
    d_index, og_ds = compute_d_index_global(data, (400, 400),  [2, 2, 2, 2, 2, 2, 2, 2, 2])
    for type_id, row in enumerate(d_index):
        indices = pd.concat([indices, pd.DataFrame({'filename': [filename], 'init_locs': [init_locs[idx]], 'seed': [init_seed[idx]], 'type_id': [type_id], '0': [row[0]], '1': [row[1]], '2': [row[2]], '3': [row[3]], '4': [row[4]], '5': [row[5]], '6': [row[6]], '7': [row[7]], '8': [row[8]]})])
    d_indices.append(d_index)

indices.to_csv('known_d_indices_global_r_2.csv', index=False)
arr = np.asarray(d_indices)
np.save('d_indices_global_r_2.npy', arr)

In [ ]:
def plot_d_index_heatmaps(D_matrices):
    """
    Plot the D-index heatmaps for each agent type.
    
    Parameters:
    - D_matrices: List of 9 matrices, each representing the D-index heatmap for a specific type.
    """
    num_types = len(D_matrices)
    
    # Create a figure with 3x3 subplots
    fig, axes = plt.subplots(3, 3, figsize=(15, 15))
    
    # Flatten axes array for easy iteration
    axes = axes.flatten()
    
    # Plot each D-index matrix
    for t in range(num_types):
        ax = axes[t]
        cax = ax.imshow(D_matrices[t].T, cmap='viridis', origin='lower')
        ax.set_title(f'{organism_group_labels[t]}')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        fig.colorbar(cax, ax=ax, orientation='vertical')
    
    plt.suptitle(f'D-index Heatmaps per Agent Type', fontsize=16)
    plt.tight_layout(rect=[0, 0, 0.95, 0.96])  # Adjust to make room for the suptitle
    plt.show()

In [ ]:
d_indices = np.load('d_indices_global_r_org.npy')
plot_d_index_heatmaps(d_indices[0])

In [ ]:
d_indices = np.load('d_indices_global_r_2.npy')
plot_d_index_heatmaps(d_indices[0])